In [8]:
import pandas as pd

# 로컬 JSON 파일 읽기
df = pd.read_json("./data/documents.jsonl", lines=True)
df.tail()


,docid,src,content
4267,ae28101b-a42e-45b7-b24b-4ea0f1fb2d50,ko_ai2_arc__ARC_Challenge__train,비뇨기계와 순환계는 혈액이 신장을 통과하면서 폐기물과 물이 제거될 때 관여하는 두 ...
4268,eb727a4f-29c7-4d0c-b364-0e67de1776e9,ko_ai2_arc__ARC_Challenge__train,로봇은 현대 산업에서 많은 역할을 수행할 수 있습니다. 그러나 로봇 사용의 중대한 ...
4269,0c8c0086-c377-4201-81fa-25159e5435a7,ko_mmlu__human_sexuality__test,"월경은 여성의 생리주기에 따라 발생하는 현상으로, 에스트로겐과 프로게스테론 수치의 ..."
4270,06da6a19-ec78-404e-9640-9fc33f63c6a2,ko_ai2_arc__ARC_Challenge__train,식물이 내뿜는 가스는 산소입니다. 식물은 광합성 과정을 통해 태양 에너지를 이용하여...
4271,03c36d5e-c711-4dc2-b4db-aaeb94d86395,ko_mmlu__computer_security__test,"버퍼 오버런은 개발자들이 만든 널리 퍼져 있는 앱의 코딩 오류로써, 공격자가 시스템..."


In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

# 1. 프롬프트 템플릿 정의
SCIENCE_5W1H_TEMPLATE = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.

문서: {document}

응답 형식은 반드시 다음과 같이 해주세요:
**1차 요약**: [문서의 길이가 100자 이내라면 원문 그대로 사용 또는 400자 내외 요약]
"""


# 2. 프롬프트 객체 생성
science_prompt = PromptTemplate(
    input_variables=["document"],
    template=SCIENCE_5W1H_TEMPLATE
)

# 3. LLM 모델 선택 (Ollama 또는 OpenAI)
# llm = Ollama(model="llama3.1:8b")  # 로컬 모델
llm = ChatOllama(model="alibayram/Qwen3-30B-A3B-Instruct-2507")

# 4. 출력 파서 (문자열)
output_parser = StrOutputParser()

# 5. LCEL 체인 구성
science_summary_chain = (
    science_prompt 
    | llm 
    | output_parser
)

In [10]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama import ChatOllama

from langchain_core.output_parsers import BaseOutputParser

class SummaryOutputParser(BaseOutputParser):
    def parse(self, text: str) -> str:
        # "요약:" 라벨 제거
        if text.startswith("요약:"):
            return text[3:].strip()
        elif "요약:" in text:
            return text.split("요약:")[-1].strip()
        return text.strip()

first_summary = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 아래 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 전체 요약길이는 문서길이의 절반내외로 한다.
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.
        
문서: {document}

응답 형식은 반드시 다음과 같이 해주세요:
요약:"""

second_summary = """
과학문서 분석 전문가로서 육하원칙에 따라 체계적으로 요약해주세요.
    - 아래 문서의 길이가 100자 이내라면 원문 그대로 사용한다.
    - 전체 요약길이는 문서길이의 절반내외로 한다. 
    - 모든문장에는 육하원칙 6가지 요소가 반드시 포함되어야 한다.
    - 각 문장에는 원문에 포함된 핵심키워드 2개이상이 사용되어야 한다.

문서: {first_summary}

응답 형식은 반드시 다음과 같이 해주세요:
요약:"""

first_prompt = PromptTemplate(
    input_variables=["document"],
    template=first_summary
)


second_prompt = PromptTemplate(
    input_variables=["document"],
    template=second_summary

)

# 체인 생성 - 1차 요약 후 바로 2차 요약으로 연결
two_step_chain = (
    #{"text": RunnablePassthrough()}
    first_prompt 
    | llm 
    | SummaryOutputParser()
    #| (lambda x: print(f"1차 요약: {x}") or x) 
    | (lambda x: {"first_summary": x})
    | second_prompt
    | llm
    | SummaryOutputParser()
)



In [ ]:
def get_summary(doc_id: str, document: str):
    
    # 1. 문서 요약
    if len(document) > 100:
        summary_result = two_step_chain.invoke({"document": document})
    else:
        summary_result = document
    return summary_result

def save_summary(df, level=2):

    if level == 0:
        df.to_csv("./data/summary_ori.csv", index=False)    
    elif level == 1:
        df.to_csv("./data/summary_one.csv", index=False)    
    else:
        df.to_csv("./data/summary_two.csv", index=False)      

In [12]:
doc_id = []
content = []
summary = []
for index, data in df.iterrows():
    #if index == 5: break
    
    result = get_summary(data.docid, data.content)
    
    if index % 50 == 0:
        print("*"*50)
        print("*"*50)
        print(data.content)
        print(result)

    doc_id.append(data.docid)
    content.append(data.content)
    summary.append(result)

""" result = get_summary(df.iloc[1339].docid, df.iloc[1339].content)
print("*"*50)
print("*"*50)
print(df.iloc[1339].content)
print(result)    

doc_id.append(df.iloc[1339].docid)
content.append(df.iloc[1339].content)
summary.append(result) """

data = {
    'docid': doc_id,
    'content': content,
    'summary': summary
}

save_summary(pd.DataFrame(data), 2)



**************************************************
**************************************************
건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.
건강한 사람은 1-2주 동안 식단 조절과 운동을 통해 에너지 균형을 유지해야 하며, 이는 칼로리 섭취와 에너지 소비의 수학적 동등성에 기반한다. 적절한 칼로리 섭취와 영양가 있는 식품, 에너지 소비 촉진 운동을 통해 근육 강화와 비만 예방이 가능하다.
**************************************************
**************************************************
움직이는 물을 이용해 전기를 생산하는 것은 많은 이점이 있지만, 단점도 존재합니다. 그 중 하나는 지역 생태계가 혼란될 수 있다는 것입니다. 물을 이용한 전기 생산은 보통 대규모 수력 발전소에서 이루어지는데, 이로 인해 물의 흐름이 변화하고 수중 생태계에 영향을 줄 수 있습니다. 또한, 수력 발전소를 건설하기 위해 수많은 토지와 자연 환경을 희생해야 할 수도 있습니다. 이는 지역 생태계에 큰 영향을 미칠 수 있으며, 생태계의 균형을 깨뜨릴 수도 있습니다. 따라서, 움직이는 물을 이용해